# Постановка задачи

### Исходные данные: 
1. Даны функции максимальной суточной производительности по двум подземным хранилищам газа (ПХГ).
2. Каждое ПХГ заполнено на 90% (файл 1.Кривые_ПХГ.xlsx). 
3. Имеется прогноз будущей потребности в ресурсе (отборах) из всех ПХГ в совокупности (2.Прогноз_ПХГ.csv).

### Задание: 
1) На период прогноза требуется найти оптимальное распределение отборов газа из каждого ПХГ такое, чтобы:
- Суммарная производительность по всем ПХГ была бы максимальной на 01.12.2021.
- Совокупные суточные отборы полностью покрывали бы потребность.
- Суточные отборы по каждому ПХГ не превышали бы максимальную производительность.
2) Опишите подходы к решению задачи, когда потребность в ресурсе из ПХГ – случайный процесс.

# Формулировка модели

#### Множества / Sets:

$W$ – подземные хранилища газа (ПХГ) / storages

$T$ – периоды планирования / days

#### Параметры / Parameters:

$pV_{t, w}$ – изначальная заполненность ПХГ / initial supply at storage $w$ at day $t$ = 1

$pD_{t}$ – потребность в ресурсе в день $t$ / demand by day $t$

$pFirstDay$ – первый период планирования / first element in $T$ set

$pLastDay$ – последний период планирования / last element in $T$ set

#### Переменные / Variables:

$vP_{t, w}$ – продуктивность ПХГ $w$ в день $t$ / productivity per $w$ at day $t$ 

$vV_{t, w}$ – объем газа в ПХГ $w$ в день $t$ / gas volume per $w$ at day $t$

##### Переменные для первого дня / Variables for first day:

$$vP_{t, w} ≤ 5 + 0.0000001 * pV_{t, w}^3 + 0.00004 * pV_{t, w}^2 - 0.02 * pV_{t, w} \forall w = 1 ∪ t = pFirstDay$$

$$vP_{t, w} ≤ 10 - 0.00006 * pV_{t, w}^2 + 0.05 * pV_{t, w} \forall w=2 ∪ t = pFirstDay$$

##### Переменные для последующих дней  / Variables for next days:

$$vP_{t, w} ≤ 5 + 0.0000001* vV_{t-1, w}^3 + 0.00004 * vV_{t-1, w}^2 - 0.02 * vV_{t-1, w} \forall w = 1 ∪ pFirstDay < t < pLastDay $$

$$vP_{t, w} ≤ 10 - 0.00006 * vV_{t-1, w}^2 + 0.05 * vV_{t-1, w} \forall w = 2 ∪ pFirstDay < t < pLastDay $$

##### Переменные для последнего дня / Variables for last day:

$$vP_{t, w} = 5 + 0.0000001 * vV_{t, w}^3 + 0.00004 * vV_{t, w}^2 - 0.02 * vV_{t, w} \forall w = 1 ∪ t = pLastDay$$

$$vP_{t, w} = 10 - 0.00006 * vV_{t, w}^2 + 0.05 * vV_{t, w} \forall w = 2 ∪ t = pLastDay$$

#### Ограничения / Constraints:

##### Балансовые огранчения / Balance constraints:
$$vV_{t, w} = pV_{t, w} - vP_{t, w} \forall t = pFirstDay$$

$$vV_{t, w} = vV_{t-1, w} - vP_{t, w}  \forall t <> pFirstDay$$

##### Ограничение потребности / Demand constraint:
$$\sum_{w} vP_{t, w} = pD_{t}$$

# Решение

Импортируем библиотеки

In [1]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

Читаем данные в датафреймы

In [2]:
path_xlsx = r"03 Data 01.xlsx"
path_csv = r"03 Data 02.csv"

df_xlsx = pd.read_excel(path_xlsx);
df_csv = pd.read_csv(path_csv);

Начинаем писать модель

In [3]:
model = pyo.ConcreteModel('Gas demand problem')

## Множества

In [4]:
model.s_warehouses = pyo.Set(initialize = ['Tank 1', 'Tank 2'])

In [5]:
model.s_days = pyo.Set(initialize = df_csv.set_index(['Дата']).index)

### Подмножества для переменных

In [6]:
model.ss_FirstWarehouse = pyo.Set(initialize = [model.s_warehouses.first()])
model.ss_SecondWarehouse = pyo.Set(initialize = [model.s_warehouses.last()])

model.ss_FirstDay = pyo.Set(initialize = [model.s_days.first()])
model.ss_LastDay = pyo.Set(initialize = [model.s_days.last()])
model.ss_OtherDays = pyo.Set(initialize = model.s_days ^ model.ss_FirstDay ^ model.ss_LastDay)

## Параметры

Объявляем стартовые объемы ПХГ

In [7]:
model.p_initialSupply = pyo.Param(model.s_warehouses, initialize = {'Tank 1': 450, 'Tank 2': 450})

Объявляем потребность по дням

In [8]:
model.p_demand = pyo.Param(model.s_days, initialize = df_csv.set_index(['Дата'])['Прогноз отборов'].to_dict())

## Переменные

Объемы

In [9]:
model.v_Volume = pyo.Var(model.s_warehouses, model.s_days, domain=pyo.NonNegativeReals)

Продуктивность

In [10]:
model.v_Productivity = pyo.Var(model.s_warehouses, model.s_days, domain=pyo.NonNegativeReals)

## Ограничения

In [11]:
def Productivity_FirstDay_Tank1(model, w, t):
    return model.v_Productivity[w, t] <= 5 + 0.0000001 * model.p_initialSupply[w]**3 + 0.00004 * model.p_initialSupply[w]**2 - 0.02 * model.p_initialSupply[w]
model.c_Productivity_FirstDay_Tank1 = pyo.Constraint(model.ss_FirstWarehouse, model.ss_FirstDay, rule=Productivity_FirstDay_Tank1)

In [12]:
def Productivity_FirstDay_Tank2(model, w, t):
    return model.v_Productivity[w, t] <= 10 - 0.00006 * model.p_initialSupply[w]**2 + 0.05 * model.p_initialSupply[w]
model.c_Productivity_FirstDay_Tank2 = pyo.Constraint(model.ss_SecondWarehouse, model.ss_FirstDay, rule=Productivity_FirstDay_Tank2)

In [13]:
def Productivity_OtherDays_Tank1(model, w, t):
    return model.v_Productivity[w, t] <= 5 + 0.0000001 * model.v_Volume[w, t]**3 + 0.00004 * model.v_Volume[w, t]**2 - 0.02 * model.v_Volume[w, t]
model.c_Productivity_OtherDays_Tank1 = pyo.Constraint(model.ss_FirstWarehouse, model.ss_OtherDays, rule=Productivity_OtherDays_Tank1)

In [14]:
def Productivity_OtherDays_Tank2(model, w, t):
    return model.v_Productivity[w, t] <= 10 - 0.00006 * model.v_Volume[w, t]**2 + 0.05 * model.v_Volume[w, t]
model.c_Productivity_OtherDays_Tank2 = pyo.Constraint(model.ss_SecondWarehouse, model.ss_OtherDays, rule=Productivity_OtherDays_Tank2)

In [15]:
def Productivity_LastDay_Tank1(model, w, t):
    return model.v_Productivity[w, t] == 5 + 0.0000001 * model.v_Volume[w, t]**3 + 0.00004 * model.v_Volume[w, t]**2 - 0.02 * model.v_Volume[w, t]
model.c_Productivity_LastDay_Tank1 = pyo.Constraint(model.ss_FirstWarehouse, model.ss_LastDay, rule=Productivity_LastDay_Tank1)

In [16]:
def Productivity_LastDay_Tank2(model, w, t):
    return model.v_Productivity[w, t] == 10 - 0.00006 * model.v_Volume[w, t]**2 + 0.05 * model.v_Volume[w, t]
model.c_Productivity_LastDay_Tank2 = pyo.Constraint(model.ss_SecondWarehouse, model.ss_LastDay, rule=Productivity_LastDay_Tank2)

In [17]:
def BalanceConstraint(model, w, t):
    if t in model.ss_FirstDay: 
        return model.v_Volume[w, t] == model.p_initialSupply[w] - model.v_Productivity[w, t]
    else:
        return model.v_Volume[w, t] == model.v_Volume[w, model.s_days.prev(t)] - model.v_Productivity[w, t]
model.c_Balance = pyo.Constraint(model.s_warehouses, model.s_days, rule=BalanceConstraint)

In [18]:
def ProductivityConstraint(model, t):
    return sum(model.v_Productivity[w, t] for w in model.s_warehouses) == model.p_demand[t]
model.c_Productivity = pyo.Constraint(model.s_days, rule=ProductivityConstraint)

### Dummy-переменная

In [19]:
model.v_dummy = pyo.Var(domain = pyo.NonNegativeReals, bounds = (0, 0))

## Целевая функция и запуск расчета

In [20]:
model.objective = pyo.Objective(expr = 1 * model.v_dummy, sense=pyo.minimize)

In [21]:
SolverFactory('ipopt').solve(model, tee = True) 

Ipopt 3.11.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:      242
Number of nonzeros in inequality constraint Jacobian.:      114
Number of nonzeros in Lagrangian Hessian.............:       58

Total number of variables............................:      120
                     variables with only lower bounds:      120
                variables with lower and upper bounds:        0


model.name="Gas demand problem";
    - termination condition: infeasible
    - message from solver: Ipopt 3.11.1\x3a Converged to a locally infeasible
      point. Problem may be infeasible.


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 150, 'Number of variables': 121, 'Sense': 'unknown'}], 'Solver': [{'Status': 'warning', 'Message': 'Ipopt 3.11.1\\x3a Converged to a locally infeasible point. Problem may be infeasible.', 'Termination condition': 'infeasible', 'Id': 200, 'Error rc': 0, 'Time': 0.9321415424346924}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

Ожидаемо ноль

In [22]:
model.objective()

0.0

In [23]:
model.display()

Model Gas demand problem

  Variables:
    v_Volume : Size=60, Index=v_Volume_index
        Key                      : Lower : Value              : Upper : Fixed : Stale : Domain
        ('Tank 1', '2021-11-01') :     0 : 445.27563347201254 :  None : False : False : NonNegativeReals
        ('Tank 1', '2021-11-02') :     0 : 438.50276325388245 :  None : False : False : NonNegativeReals
        ('Tank 1', '2021-11-03') :     0 :  432.5918299578894 :  None : False : False : NonNegativeReals
        ('Tank 1', '2021-11-04') :     0 : 427.07515619111723 :  None : False : False : NonNegativeReals
        ('Tank 1', '2021-11-05') :     0 :  422.3058509362075 :  None : False : False : NonNegativeReals
        ('Tank 1', '2021-11-06') :     0 :  417.7957143397581 :  None : False : False : NonNegativeReals
        ('Tank 1', '2021-11-07') :     0 : 413.91326367752316 :  None : False : False : NonNegativeReals
        ('Tank 1', '2021-11-08') :     0 :   410.501089638594 :  None : False : False 

# Рассуждения про стохастику



Нужно добавить еще одно множество сценариев: оптимистичный, пессимистичный и базовый. Добавить индекс сценария в параметр потребности в день и вероятность сценария. Индекс пойдет в переменные потребности, балансовые ограничения и ограничения потребности.  При расчете получим дерево решений.